In [1]:
import config 
import dataset
from patchify import patchify,unpatchify
import cv2
from model import UNet
import utils
import torch
import torchvision
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from torchview import draw_graph
import graphviz
import os 
graphviz.set_jupyter_format('png')

/home/gizmoo/anaconda3/envs/tork/lib/python3.9/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


'svg'

In [2]:
img_path = "/home/gizmoo/Desktop/Seg/Masks/"
img_patches_path = "/home/gizmoo/Desktop/Seg/Masks_patches/"
patch_size = (256,256)

In [3]:
#utils.generate_mask_patches(img_path,img_patches_path,patch_size)
#utils.generate_image_patches(mask_path,mask_patches_path,patch_size)
#utils.concatenate_images(img_patches_path,mask_patches_path,output_path)


In [4]:
def corp_image(image,patch_size):
    patch_size_x , patch_size_y,channels = patch_size
    if patch_size_x > image.shape[1] or patch_size_y > image.shape[0]:
        raise ValueError("Patch size is larger than image size")
    SIZE_X = (image.shape[1]//patch_size_x)*patch_size_x
    SIZE_Y = (image.shape[0]//patch_size_y)*patch_size_y
    rest_pixels_x = image.shape[1] - SIZE_X
    rest_pixels_y = image.shape[0] - SIZE_Y
    origin_x = rest_pixels_x // 2
    origin_y = rest_pixels_y // 2
    corped_image = image[origin_y:origin_y+SIZE_Y, origin_x:origin_x+SIZE_X]
    return corped_image

def denormalize(tensor, mean, std):
    for t, m, s in zip(tensor, mean, std):
        t = t*s + m
    return tensor


# def make_prediction(image_patches,model, optimizer ,checkpoint_file):
#     image_prediction = []
#     utils.load_checkpoint(checkpoint_file,model = model ,optimizer=optimizer,lr  = config.LEARNING_RATE)
#     #model.load_state_dict(torch.load(weights)[0])
#     model.eval()
#     for i in range(image_patches.shape[0]):
#         for j in range(image_patches.shape[1]):
#             patch = image_patches[i,j,:,:]
#             patch = patch.squeeze()
#             print(patch.shape)
#             patch = config.val_transforms(image = patch)["image"]
#             print(patch.shape)
            
#             with torch.no_grad():
#                 patch = patch.unsqueeze(0).to(device = config.DEVICE) 
#                 prediction = torch.sigmoid(model(patch))
#                 prediction = (prediction > 0.5).float()
#                 #print(prediction)
#                 image_prediction.append(prediction.cpu().numpy())
#     image_prediction = np.array(image_prediction).squeeze(1)
#     image_prediction = image_prediction.reshape(image_patches.shape[0],image_patches.shape[1],patch_size[0], patch_size[1])
#     target_image_shape = (512,512)
#     print(image_prediction.shape)

#     merged_image = unpatchify(image_prediction,target_image_shape) 
#     print(merged_image.shape)   
#     return merged_image



# image_path = "/home/gizmoo/Desktop/Seg/Bface/Blockface_RGB_66x66x60um_s841-s1083_234s0047.png"
# image = cv2.imread(image_path,cv2.IMREAD_COLOR)
# image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
# print(image.shape)
# patch_size = (256,256,3)
# corped_image = corp_image(image,patch_size)
# print(corped_image.shape)
# image_patches = patchify(corped_image,patch_size,step = patch_size[0])
# print(image_patches.shape)
# model = UNet(in_channels=3 , out_channels=1).to(device = config.DEVICE)
# optimizer = torch.optim.Adam(model.parameters(),lr =config.LEARNING_RATE)
# merged_image = make_prediction( image_patches, model,optimizer ,checkpoint_file = "/home/gizmoo/Desktop/MA-code/U_Net/saves/save_20240110_134901/unet.pth.tar" )
# #print(predicted_image.shape)
# merged_image = denormalize(merged_image, [0.0, 0.0, 0.0], [1.0, 1.0, 1.0])
# merged_image = merged_image * 255
# merged_image = merged_image.astype(np.uint8)
# plt.imshow(merged_image)

In [5]:
patch_size = (256,256,3)
image_path = "/home/gizmoo/Desktop/Seg/Bface/Blockface_RGB_66x66x60um_s841-s1083_234s0047.png"
checkpoint_file = "/home/gizmoo/Desktop/MA-code/U_Net/saves/save_20240110_134901/unet.pth.tar"
save_image = True
save_dir = "/home/gizmoo/Desktop/results"
def make_UNet_prediction(image_path, patch_size,checkpoint_file,save_images = False, save_dir =None):
    image = cv2.imread(image_path,cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    corped_image = corp_image(image,patch_size)
    image_patches = patchify(corped_image,patch_size,step = patch_size[0])
    model = UNet(in_channels=3 , out_channels=1).to(device = config.DEVICE)
    optimizer = torch.optim.Adam(model.parameters(),lr =config.LEARNING_RATE)
    image_prediction = []
    utils.load_checkpoint(checkpoint_file,model = model ,optimizer=optimizer,lr  = config.LEARNING_RATE)
    #model.load_state_dict(torch.load(weights)[0])
    model.eval()
    for i in range(image_patches.shape[0]):
        for j in range(image_patches.shape[1]):
            patch = image_patches[i,j,:,:]
            patch = patch.squeeze()
            patch = config.val_transforms(image = patch)["image"]
            with torch.no_grad():
                patch = patch.unsqueeze(0).to(device = config.DEVICE) 
                prediction = torch.sigmoid(model(patch))
                prediction = (prediction > 0.5).float()
                image_prediction.append(prediction.cpu().numpy())
    image_prediction = np.array(image_prediction).squeeze(1)
    image_prediction = image_prediction.reshape(image_patches.shape[0],image_patches.shape[1],patch_size[0], patch_size[1])
    target_image_shape = (corped_image.shape[0],corped_image.shape[1])
    merged_image = unpatchify(image_prediction,target_image_shape)
    merged_image = denormalize(merged_image, [0.0, 0.0, 0.0], [1.0, 1.0, 1.0])
    merged_image = merged_image * 255
    merged_image = merged_image.astype(np.uint8)
    if save_images and save_dir is not None:
        # Ensure the save directory exists
        os.makedirs(save_dir, exist_ok=True)

        # Save the cropped image as an RGB image
        cropped_image_path = os.path.join(save_dir, 'cropped_image.png')
        cv2.imwrite(cropped_image_path, cv2.cvtColor(corped_image, cv2.COLOR_RGB2BGR))

        # Save the merged prediction image as a grayscale image
        merged_image_path = os.path.join(save_dir, 'merged_image.png')
        cv2.imwrite(merged_image_path, merged_image)

    return corped_image, merged_image


x,y = make_UNet_prediction(image_path=image_path, patch_size=patch_size, checkpoint_file= checkpoint_file, save_images=True, save_dir= save_dir)

=> Loading checkpoint
